<a href="https://colab.research.google.com/github/nhnain/eegchallenge/blob/main/Challenge1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Preprocessing

In [1]:
!pip install braindecode
!pip install eegdash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.2/305.2 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 121.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.9/168.9 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.1/263.1 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 141.0 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
dask-cudf-cu12 25.6

In [8]:
from pathlib import Path
import math
import os
import random
from joblib import Parallel, delayed

import torch
from torch.utils.data import DataLoader
from torch import optim
from torch.nn.functional import l1_loss
from braindecode.preprocessing import preprocess, Preprocessor, create_fixed_length_windows, create_windows_from_events
from braindecode.datasets.base import EEGWindowsDataset, BaseConcatDataset, BaseDataset
from braindecode.models import EEGNeX
from eegdash import EEGChallengeDataset
from typing import Optional
from torch.nn import Module
from torch.optim.lr_scheduler import LRScheduler
from tqdm import tqdm
from eegdash.dataset import EEGChallengeDataset
from eegdash.hbn.windows import annotate_trials_with_target, add_aux_anchors, add_extras_columns, keep_only_recordings_with

In [9]:
# Identify whether a CUDA-enabled GPU is available
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    msg ='CUDA-enabled GPU found. Training should be faster.'
else:
    msg = (
        "No GPU found. Training will be carried out on CPU, which might be "
        "slower.\n\nIf running on Google Colab, you can request a GPU runtime by"
        " clicking\n`Runtime/Change runtime type` in the top bar menu, then "
        "selecting \'T4 GPU\'\nunder \'Hardware accelerator\'."
    )
print(msg)

CUDA-enabled GPU found. Training should be faster.


In [10]:
release_list = ['R1', 'R2', 'R3', 'R4', 'R5', 'R6',
                'R7', 'R8', 'R9', 'R10', 'R11']
train_set = []
valid_set = []
test_set = []

DATA_DIR = Path('data')
DATA_DIR.mkdir(parents=True, exist_ok=True)


In [12]:
for release in tqdm(release_list):
  dataset_ccd = EEGChallengeDataset(
      task = 'contrastChangeDetection',
      release = release,
      cache_dir = DATA_DIR,
      mini = True
  )
  raws = Parallel(n_jobs=-1)(
      delayed(lambda d: d.raw)(d) for d in dataset_ccd.datasets
  )

  EPOCH_LEN_S = 2.0
  SFREQ = 100

  transformation_offline = [
      Preprocessor(
          annotate_trials_with_target,
          target_field = 'rt_from_stimulus', epoch_length =  EPOCH_LEN_S,
          require_stimulus = True,
          require_response = True,
          apply_on_array = False,
      ),
      Preprocessor(add_aux_anchors, apply_on_array=False),
  ]
  preprocess (dataset_ccd, transformation_offline, n_jobs=1)

  ANCHOR = 'stimulus_anchor'
  SHIFT_AFTER_STIM = 0.5
  WINDOW_LEN = 2.0

  dataset = keep_only_recordings_with(ANCHOR, dataset_ccd)

  single_windows = create_windows_from_events(
      dataset,
      mapping={ANCHOR: 0},
      trial_start_offset_samples = int(SHIFT_AFTER_STIM * SFREQ),
      trial_stop_offset_samples = int((SHIFT_AFTER_STIM + WINDOW_LEN) * SFREQ),
      window_size_samples = int(EPOCH_LEN_S * SFREQ),
      window_stride_samples = SFREQ,
      preload = True,
  )

  single_windows = add_extras_columns(
      single_windows,
      dataset,
      desc = ANCHOR, keys = ("target", "rt_from_stimulus", "rt_from_trialstart",
            "stimulus_onset", "response_onset", "correct", "response_type"))

  meta_information = single_windows.get_metadata()

  subjects = meta_information['subject'].unique()
  sub_rm = ["NDARWV769JM7", "NDARME789TD2", "NDARUA442ZVF", "NDARJP304NK1",
            "NDARTY128YLU", "NDARDW550GU6", "NDARLD243KRE", "NDARUJ292JXV", "NDARBA381JGH"]
  subjects = [s for s in subjects if s not in sub_rm]

  subject_split = single_windows.split('subject')

  if release != 'R5':
    for s in subject_split:
      train_set.append(subject_split[s])
  else:
    for s in subject_split:
      valid_set.append(subject_split[s])

  0%|          | 0/11 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/eegdash/dataset/dataset.py:126: UserWarning: 

[EEGChallengeDataset] EEG 2025 Competition Data Notice:
-------------------------------------------------------
This object loads the HBN dataset that has been preprocessed for the EEG Challenge:
  - Downsampled from 500Hz to 100Hz
  - Bandpass filtered (0.5–50 Hz)

For full preprocessing details, see:
  https://github.com/eeg2025/downsample-datasets

IMPORTANT: The data accessed via `EEGChallengeDataset` is NOT identical to what you get from `EEGDashDataset` directly.
If you are participating in the competition, always use `EEGChallengeDataset` to ensure consistency with the challenge data.


  warn(


Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_(

  9%|▉         | 1/11 [00:14<02:28, 14.85s/it]/usr/local/lib/python3.12/dist-packages/eegdash/dataset/dataset.py:126: UserWarning: 

[EEGChallengeDataset] EEG 2025 Competition Data Notice:
-------------------------------------------------------
This object loads the HBN dataset that has been preprocessed for the EEG Challenge:
  - Downsampled from 500Hz to 100Hz
  - Bandpass filtered (0.5–50 Hz)

For full preprocessing details, see:
  https://github.com/eeg2025/downsample-datasets

IMPORTANT: The data accessed via `EEGChallengeDataset` is NOT identical to what you get from `EEGDashDataset` directly.
If you are participating in the competition, always use `EEGChallengeDataset` to ensure consistency with the challenge data.


  warn(


Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_(

 18%|█▊        | 2/11 [03:54<20:19, 135.53s/it]/usr/local/lib/python3.12/dist-packages/eegdash/dataset/dataset.py:126: UserWarning: 

[EEGChallengeDataset] EEG 2025 Competition Data Notice:
-------------------------------------------------------
This object loads the HBN dataset that has been preprocessed for the EEG Challenge:
  - Downsampled from 500Hz to 100Hz
  - Bandpass filtered (0.5–50 Hz)

For full preprocessing details, see:
  https://github.com/eeg2025/downsample-datasets

IMPORTANT: The data accessed via `EEGChallengeDataset` is NOT identical to what you get from `EEGDashDataset` directly.
If you are participating in the competition, always use `EEGChallengeDataset` to ensure consistency with the challenge data.


  warn(


...

 82%|████████▏ | 9/11 [28:21<06:48, 204.00s/it]/usr/local/lib/python3.12/dist-packages/eegdash/dataset/dataset.py:126: UserWarning: 

[EEGChallengeDataset] EEG 2025 Competition Data Notice:
-------------------------------------------------------
This object loads the HBN dataset that has been preprocessed for the EEG Challenge:
  - Downsampled from 500Hz to 100Hz
  - Bandpass filtered (0.5–50 Hz)

For full preprocessing details, see:
  https://github.com/eeg2025/downsample-datasets

IMPORTANT: The data accessed via `EEGChallengeDataset` is NOT identical to what you get from `EEGDashDataset` directly.
If you are participating in the competition, always use `EEGChallengeDataset` to ensure consistency with the challenge data.


  warn(


Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_(

 91%|█████████ | 10/11 [31:40<03:22, 202.42s/it]/usr/local/lib/python3.12/dist-packages/eegdash/dataset/dataset.py:126: UserWarning: 

[EEGChallengeDataset] EEG 2025 Competition Data Notice:
-------------------------------------------------------
This object loads the HBN dataset that has been preprocessed for the EEG Challenge:
  - Downsampled from 500Hz to 100Hz
  - Bandpass filtered (0.5–50 Hz)

For full preprocessing details, see:
  https://github.com/eeg2025/downsample-datasets

IMPORTANT: The data accessed via `EEGChallengeDataset` is NOT identical to what you get from `EEGDashDataset` directly.
If you are participating in the competition, always use `EEGChallengeDataset` to ensure consistency with the challenge data.


  warn(


Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_('stimulus_anchor')]
Used Annotations descriptions: [np.str_(

100%|██████████| 11/11 [34:50<00:00, 190.07s/it]


In [13]:
train_set = BaseConcatDataset(train_set)
valid_set = BaseConcatDataset(valid_set)

print(f'Train set: {len(train_set)}')
print(f'Valid set: {len(valid_set)}')

Train set: 12805
Valid set: 1214


#Build model

In [21]:
batch_size = 128
num_workers = 1

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=num_workers)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [22]:
from braindecode.models.util import models_dict

names = sorted(models_dict)
w = max(len(n) for n in names)

for i in range(0, len(names), 3):
    row = names[i:i+3]
    print("  ".join(f"{n:<{w}}" for n in row))

ATCNet                  AttentionBaseNet        AttnSleep             
BDTCN                   BIOT                    CTNet                 
ContraWR                Deep4Net                DeepSleepNet          
EEGConformer            EEGITNet                EEGInceptionERP       
EEGInceptionMI          EEGMiner                EEGNeX                
EEGNet                  EEGSimpleConv           EEGTCNet              
FBCNet                  FBLightConvNet          FBMSNet               
IFNet                   Labram                  MSVTNet               
SCCNet                  SPARCNet                ShallowFBCSPNet       
SignalJEPA              SignalJEPA_Contextual   SignalJEPA_PostLocal  
SignalJEPA_PreLocal     SincShallowNet          SleepStagerBlanco2020 
SleepStagerChambon2018  SyncNet                 TIDNet                
TSception               USleep                


In [24]:
model = EEGNeX(n_chans=129,
               n_outputs=1,
               n_times=200,
               sfreq=100)

#Model training

In [25]:
lr = 1e-3
weight_decay = 1e-5
n_epochs = 100
early_stopping_patience = 50

In [34]:
def train_one_epoch(
    dataloader: DataLoader,
    model: Module,
    loss_fn,
    optimizer,
    scheduler: Optional[LRScheduler],
    epoch: int,
    device,
    print_batch_stats: bool = True,):
  model.train()

  total_loss = 0.0
  sum_sq_err = 0.0
  n_samples = 0

  progress_bar = tqdm(
      enumerate(dataloader),
      total=len(dataloader),
      disable=not print_batch_stats)
  for batch_idx, batch in progress_bar:
    X, y = batch[0], batch[1]
    X, y = X.to(device).float(), y.to(device).float()

    optimizer.zero_grad(set_to_none=True)
    preds = model(X)
    loss = loss_fn(preds, y)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

    preds_flat = preds.detach().view(-1)
    y_flat = y.detach().view(-1)
    sum_sq_err += torch.sum((preds_flat - y_flat)**2).item()
    n_samples += y_flat.numel()

    if print_batch_stats:
      running_rmse = (sum_sq_err / max(n_samples, 1)) ** 0.5
      progress_bar.set_description(
          f"Epoch {epoch}, Batch {batch_idx + 1}/{len(dataloader)},"
          f"Loss: {loss.item():.6f}, RMSE: {running_rmse:.6f}"
      )
  if scheduler is not None:
    scheduler.step()

  avg_loss = total_loss/len(dataloader)
  rmse = (sum_sq_err/max(n_samples, 1))** 0.5
  return avg_loss, rmse

In [35]:
with torch.inference_mode():
  def valid_model(
      dataloader: DataLoader,
      model: Module,
      loss_fn,
      device,
      print_batch_stats: bool = True):

    model.eval()

    total_loss = 0.0
    sum_sq_err = 0.0
    n_batches = len(dataloader)
    n_samples = 0.0

    iterator = tqdm(
        enumerate(dataloader),
        total=len(dataloader),
        disable=not print_batch_stats)

    for batch_idx, batch in iterator:
      X, y = batch[0], batch[1]
      X, y = X.to(device).float(), y.to(device).float()

      preds = model(X)
      batch_loss = loss_fn(preds, y).item()
      total_loss += batch_loss

      preds_flat = preds.detach().view(-1)
      y_flat = y.detach().view(-1)
      sum_sq_err += torch.sum((preds_flat - y_flat)**2).item()
      n_samples += y_flat.numel()

      if print_batch_stats:
        running_rmse = (sum_sq_err/max(n_samples,1)) ** 0.5
        iterator.set_description(
            f"Val Batch {batch_idx + 1}/{len(dataloader)},"
            f"Loss: {batch_loss:.6f}, RMSE: {running_rmse:.6f}"
        )

    avg_loss = total_loss / n_batches if n_batches else float("nan")
    rmse = (sum_sq_err / max(n_samples, 1)) ** 0.5

    print(f"Val RMSE: {rmse:.6f}, Val Loss: {avg_loss:.6f}")
    return avg_loss, rmse

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = n_epochs -1)
loss_fn = torch.nn.MSELoss()

patience = 5
min_delta = 1e-4
best_rmse = float('inf')
epochs_no_improve = 0
best_state, best_epoch = None, None

model.to(device)
for epoch in range(1, n_epochs+1):
  print(f'Epoch {epoch}/{n_epochs}: ', end='')

  train_loss, train_rmse = train_one_epoch(
      train_loader, model, loss_fn, optimizer, scheduler, epoch, device
  )
  valid_loss, valid_rmse = valid_model(
      valid_loader, model, loss_fn, device
  )
  print(
      f'Train RMSE: {train_rmse:.6f}, '
      f'Average Train Loss: {train_loss:.6f}, '
      f'Val RMSE: {valid_rmse:.6f}, '
      f'Average Valid Loss: {valid_loss:.6f}\n'
  )
if best_state is not None:
  model.load_state_dict(best_state)

Epoch 1/100: 

Epoch 1, Batch 101/101,Loss: 0.184482, RMSE: 0.455785: 100%|██████████| 101/101 [00:49<00:00,  2.02it/s]
Val Batch 10/10,Loss: 0.267905, RMSE: 0.405653: 100%|██████████| 10/10 [00:03<00:00,  2.98it/s]

Val RMSE: 0.405653, Val Loss: 0.169883

Train RMSE: 0.455785, Average Train Loss: 0.207519, Val RMSE: 0.405653, Average Valid Loss: 0.169883

Epoch 2/100: 


Epoch 2, Batch 101/101,Loss: 0.158314, RMSE: 0.448486: 100%|██████████| 101/101 [00:49<00:00,  2.03it/s]
Val Batch 10/10,Loss: 0.268793, RMSE: 0.413763: 100%|██████████| 10/10 [00:03<00:00,  2.96it/s]

Val RMSE: 0.413763, Val Loss: 0.176232

Train RMSE: 0.448486, Average Train Loss: 0.200732, Val RMSE: 0.413763, Average Valid Loss: 0.176232

Epoch 3/100: 

...


Epoch 65, Batch 101/101,Loss: 0.141731, RMSE: 0.384779: 100%|██████████| 101/101 [00:49<00:00,  2.05it/s]
Val Batch 10/10,Loss: 0.161905, RMSE: 0.390996: 100%|██████████| 10/10 [00:03<00:00,  3.03it/s]

Val RMSE: 0.390996, Val Loss: 0.153343

Train RMSE: 0.384779, Average Train Loss: 0.147994, Val RMSE: 0.390996, Average Valid Loss: 0.153343

Epoch 66/100: 


Epoch 66, Batch 101/101,Loss: 0.086074, RMSE: 0.385922: 100%|██████████| 101/101 [00:49<00:00,  2.05it/s]
Val Batch 10/10,Loss: 0.161922, RMSE: 0.405263: 100%|██████████| 10/10 [00:03<00:00,  3.02it/s]

Val RMSE: 0.405263, Val Loss: 0.164119

Train RMSE: 0.385922, Average Train Loss: 0.148337, Val RMSE: 0.405263, Average Valid Loss: 0.164119

Epoch 67/100: 


Epoch 67, Batch 81/101,Loss: 0.177325, RMSE: 0.385006:  80%|████████  | 81/101 [00:39<00:09,  2.21it/s]

In [ ]:
torch.save(model.state_dict(), 'model.pth')